In [1]:
import requests
import urllib.request, urllib.parse, urllib.error
import re
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

In [2]:
df_master_dataset = pd.read_csv("../Master_Dataset_Raw.csv")
# convert datetime
df_master_dataset['Account Created Date'] = pd.to_datetime(df_master_dataset['Account Created Date'])

df_master_dataset['Associated Film Festivals'] = pd.Series(dtype=str)

df_master_dataset

,Story Primary ID,Story ID,User Primary ID,User ID,Gender,Age,Title,Narrative,Media,Account Created Date,Interest,Associated Film Festivals
0,156581,STY1666112066,86625,USR1662300553,NaN,12,Qatar Airways cuts flights to accommodate FIFA...,"<p dir=""ltr"">Qatar Airways&#160;has withdrawn ...",https://image.pixstory.com/Pixstory-image-1666...,2022-09-04 19:39:13,"Technology, History, Food, Entertainment, Spor...",NaN
1,156580,STY1666111954,119236,USR1665943314,NaN,18,Positive thinking does not necessarily mean av...,"<p dir=""ltr"">Positive thinking does not necess...",https://image.pixstory.com/Pixstory-image-1666...,2022-10-16 23:31:54,"Technology, History, Food, Entertainment, Spor...",NaN
2,156579,STY1666111923,86566,USR1662294899,NaN,12,FIFA negotiating 'initiatives' for Qatar migra...,"<p dir=""ltr"">FIFA's president sought Monday to...",https://image.pixstory.com/Pixstory-image-1666...,2022-09-04 18:04:59,"Technology, History, Food, Entertainment, Spor...",NaN
3,156578,STY1666111798,86655,USR1662303432,NaN,12,Qatar ready for the World Cup in just over a m...,"<p dir=""ltr"">Ticket sales for the FIFA World C...",https://image.pixstory.com/Pixstory-image-1666...,2022-09-04 20:27:12,"Technology, History, Food, Entertainment, Spor...",NaN
4,156577,STY1666111463,9221,USR1632727815,male,23,"Froyo always makes everyone happy, make this h...","<p dir=""ltr""></p>\n<p dir=""ltr"">Almost all of ...",https://image.pixstory.com/Pixstory-image-1666...,2021-09-27 13:00:15,"Science, Inequality, Education, Technology, He...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
94995,95021,STY1650382324,22770,USR1642425119,male,40,Ronaldo salta la trasferta di Liverpool,Il Manchester United ha annunciato l'assenza d...,https://image.pixstory.com/Pixstory-image-1650...,2022-01-17 18:41:59,"calcio, premier league, cristiano ronaldo, man...",NaN
94996,95020,STY1650381837,32577,USR1650381467,NaN,15,Name-vishal\nFather name-Amod\nClass-7\nAge-13...,Earthday,https://image.pixstory.com/Pixstory-image-1650...,2022-04-19 20:47:47,chintan,NaN
94997,95019,STY1650381574,32498,USR1650370047,NaN,15,Earth day,Name Muskan \nClass 8\nSex f,https://image.pixstory.com/Pixstory-image-1650...,2022-04-19 17:37:27,chintan,NaN
94998,95018,STY1650380690,32574,USR1650380153,NaN,12,Name-puneet\nclass-6th\nfather name-Ramkishan\...,Earthday,https://image.pixstory.com/Pixstory-image-1650...,2022-04-19 20:25:53,chintan,NaN


In [3]:
# Define the input string
date_range_str = "20-30 January 2022"

# Parse the start and end dates from the input string
start_date_str, end_date_str = date_range_str.split("-")
print(start_date_str, end_date_str)
start_date_str = start_date_str + " " + end_date_str.split(" ")[1] + " " + end_date_str.split(" ")[2]
end_date_str = end_date_str.strip()
print(start_date_str, "-", end_date_str)

# Convert the start and end dates to datetime objects
start_date = datetime.strptime(start_date_str, "%d %B %Y")
end_date = datetime.strptime(end_date_str, "%d %B %Y")

# If the end date is less than the start date, assume it's in the following month
if end_date < start_date:
    end_date += timedelta(days=31)

# Print the results
print("Start date:", start_date)
print("End date:", end_date)


20 30 January 2022
20 January 2022 - 30 January 2022
Start date: 2022-01-20 00:00:00
End date: 2022-01-30 00:00:00


# Adding 2022 Film Festivals

In [4]:
# Make a request to the web page
url = 'https://www.film-fest-report.com/home/film-festivals-2022'
html = urllib.request.urlopen(url).read()

# Parse the HTML content of the page
soup = BeautifulSoup(html, 'html.parser')

# Find all <p> tags with class="preFade fadeIn"
# p_tags = soup.find_all('p', class_='preFade fadeIn')
p_tags = soup.find_all('p')

p_tags = list(p_tags)

In [5]:
# Print the text content of each <p> tag
film_festivals_list = []

for p_tag in p_tags:
    if p_tag.text != '':
        film_festivals_list.append(p_tag.text)
    # print(p_tag.text)

del film_festivals_list[0]
del film_festivals_list[0]
del film_festivals_list[118]
del film_festivals_list[118]
del film_festivals_list[118]
del film_festivals_list[118]
del film_festivals_list[34]
del film_festivals_list[62]

In [6]:
film_festivals_dict = {}

for i in range(0, len(film_festivals_list), 2):
    film_festivals_dict[film_festivals_list[i]] = film_festivals_list[i+1]

In [7]:
# Convert the dictionary to a DataFrame
df_2022 = pd.DataFrame(film_festivals_dict.items(), columns=['Festival Name', 'Raw Date'])

In [8]:
df_2022['Start Date'] = pd.Series(dtype=str)
df_2022['End Date'] = pd.Series(dtype=str)

In [9]:
# Method 1: Using iterrows()
for index, row in df_2022.iterrows():
    # print(f"Row {index}: {row['Film Festival']}, {row['Raw Date']}")

    # Parse the start and end dates from the input string
    start_date_str, end_date_str = row['Raw Date'].split("-")
    # print(start_date_str, end_date_str)

    if (row['Raw Date'][1] == "-" or row['Raw Date'][2] == "-"):
        start_date_str = start_date_str + " " + end_date_str.split(" ")[1] + " " + end_date_str.split(" ")[2]
    else:
        start_date_str = start_date_str + end_date_str.split(" ")[3]

    end_date_str = end_date_str.strip()
    # print(start_date_str, "-", end_date_str)

    # Convert the start and end dates to datetime objects
    start_date = datetime.strptime(start_date_str, "%d %B %Y")
    end_date = datetime.strptime(end_date_str, "%d %B %Y")

    row['Start Date'] = start_date
    row['End Date'] = end_date

In [10]:
del df_2022["Raw Date"]

In [11]:
df_2022

,Festival Name,Start Date,End Date
0,Sundance Film Festival 2022,2022-01-20 00:00:00,2022-01-30 00:00:00
1,International Film Festival Rotterdam 2022,2022-01-26 00:00:00,2022-02-06 00:00:00
2,Clermont-Ferrand International Short Film Fest...,2022-01-28 00:00:00,2022-02-05 00:00:00
3,Berlin International Film Festival 2022,2022-02-10 00:00:00,2022-02-20 00:00:00
4,Glasgow International Film Festival 2022,2022-03-02 00:00:00,2022-03-13 00:00:00
5,Santa Barbara International Film Festival 2022,2022-03-02 00:00:00,2022-03-12 00:00:00
6,True/False Film Fest 2022,2022-03-03 00:00:00,2022-03-06 00:00:00
7,SXSW 2022,2022-03-11 00:00:00,2022-03-20 00:00:00
8,CPH:DOX 2022,2022-03-23 00:00:00,2022-04-03 00:00:00
9,Hong Kong International Film Festival 2022,2022-03-31 00:00:00,2022-04-11 00:00:00


# Adding 2020 Film Festivals

https://www.filmfestivaldatabase.com

In [12]:
df_2020 = pd.read_csv("2020 Film Festival Database & Calendar - 2020 DATABASE.csv")

In [13]:
df_2020

,Object ID,Last Year's Month,ADDRESS,Primary Category for Calendar,Category,GA,Festival Name,Website,FilmFreeway / Submissions,Date Changes & Cancellations due to COVID-19,...,Animation,Comedy,Student,Misc. Niche,"Environment, Outdoor, Adventure, Wildlife","Social Justice, Peace",Children's,Dance,Christian,"Experimental, Video Art (etc)"
0,765.0,NaN,"1300 North Atlantic Avenue, Cocoa Beach, Flori...",Short,Short,NaN,15 Minutes of Fame Indie Film Festival,https://etheridgeproductions.com/,https://filmfreeway.com/15MinutesofFame,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,935.0,11.0,"352 Cypress Street, Abilene, Texas 79601 Unite...",Short,Short,NaN,24fps International Short Film Festival,http://24fpsfest.com,https://filmfreeway.com/24fpsfest,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,769.0,NaN,"567 Colorado Avenue, Carbondale, Colorado 81623","Environment, Outdoor, Adventure, Wildlife","Short, Environmental/Outdoor/Adventure/Wildlife",NaN,5Point Adventure Film Festival,https://5pointfilm.org/,https://filmfreeway.com/5PointFilmFestival,New dates: Oct 14-18,...,NaN,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN
3,926.0,NaN,"1101 Center Avenue Mall, Fort Collins, Colorad...",Student,"Student, Short",NaN,ACT Human Rights Film Festival,https://actfilmfest.colostate.edu,https://filmfreeway.com/ACTHumanRightsFilmFest...,Postponed,...,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,11.0,"3545 Lipan Street Denver, CO 80211 United States",Misc. Niche,Misc. Niche,NaN,Action Figure Film Festival,http://www.actionfigurefilmfestival.com/,https://filmfreeway.com/afff,NaN,...,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,892.0,10.0,"Sutton, WV United States",General,General,NaN,WV FILMmakers Festival,http://wvfilmmakersfestival.org,https://filmfreeway.com/WVFILMmakersFestival,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1018,749.0,NaN,"Yale University New Haven, CT 06520 United States",Student,Student,NaN,Yale Student Film Festival,http://www.yalestudentfilmfest.com/,https://filmfreeway.com/YSFF,NaN,...,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1019,829.0,10.0,"328 Jackson St, Columbus, IN 47201",General,General,NaN,Yes Film Festival,https://www.yesfilmfestival.com/,https://filmfreeway.com/YESFilmFestival,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1020,866.0,11.0,"540 Nepperhan Avenue Suite 556 Yonkers, New Yo...",General,General,NaN,"YoFiFest, The Yonkers Film Festival",https://www.yofifest.com/,https://filmfreeway.com/YoFiFest,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df_2020 = df_2020.loc[:, ['Festival Name', 'Start Date', 'End Date']]

In [15]:
# select rows where 'Start Date' is 'tbd' and drop them
df_2020 = df_2020.drop(df_2020[df_2020['Start Date'] == 'tbd'].index)

In [16]:
# convert 'Start Date' and 'End Date' columns to datetime objects
df_2020['Start Date'] = pd.to_datetime(df_2020['Start Date'], format='%m/%d/%y')

In [17]:
df_2020['End Date'] = pd.to_datetime(df_2020['End Date'], format='%m/%d/%y')

In [18]:
df_2020

,Festival Name,Start Date,End Date
0,15 Minutes of Fame Indie Film Festival,2020-02-29,2020-02-29
1,24fps International Short Film Festival,2020-11-06,2020-11-07
2,5Point Adventure Film Festival,2020-04-24,2020-04-26
3,ACT Human Rights Film Festival,2020-04-03,2020-04-11
5,Action On Film Festival,2020-07-23,2020-08-02
...,...,...,...
1014,"Workers Unite Film Festival, The",2020-05-08,2020-05-23
1016,Worldfest-Houston International Film & Video F...,2020-04-17,2020-04-26
1018,Yale Student Film Festival,2020-03-27,2020-03-29
1020,"YoFiFest, The Yonkers Film Festival",2020-11-06,2020-11-13


In [19]:
# Concatenate the two data frames
df_2022_2020 = pd.concat([df_2022, df_2020])

# Print the concatenated data frame
df_2022_2020

,Festival Name,Start Date,End Date
0,Sundance Film Festival 2022,2022-01-20 00:00:00,2022-01-30 00:00:00
1,International Film Festival Rotterdam 2022,2022-01-26 00:00:00,2022-02-06 00:00:00
2,Clermont-Ferrand International Short Film Fest...,2022-01-28 00:00:00,2022-02-05 00:00:00
3,Berlin International Film Festival 2022,2022-02-10 00:00:00,2022-02-20 00:00:00
4,Glasgow International Film Festival 2022,2022-03-02 00:00:00,2022-03-13 00:00:00
...,...,...,...
1014,"Workers Unite Film Festival, The",2020-05-08 00:00:00,2020-05-23 00:00:00
1016,Worldfest-Houston International Film & Video F...,2020-04-17 00:00:00,2020-04-26 00:00:00
1018,Yale Student Film Festival,2020-03-27 00:00:00,2020-03-29 00:00:00
1020,"YoFiFest, The Yonkers Film Festival",2020-11-06 00:00:00,2020-11-13 00:00:00


# Adding 2021 Film Festivals

In [20]:
# Make a request to the web page
# url = 'https://www.screendaily.com/news/2021-film-festivals-and-markets-latest-dates-postponements-and-cancellations/5155284.article'
# html_bytes = urllib.request.urlopen(url).read()

with open('2021_film_festivals_html.html', 'rb') as file:
    html_bytes = file.read()

# Parse the HTML content of the page
soup = BeautifulSoup(html_bytes, 'html.parser')

# Find all <p> tags with class="preFade fadeIn"
# p_tags = soup.find_all('p', class_='preFade fadeIn')
p_tags = soup.find_all('p')

p_tags = list(p_tags)

In [21]:
p_tags = p_tags[6:len(p_tags)]

In [22]:
film_festivals_list_raw = []

for tag in p_tags:
    film_festivals_list_raw.append(tag.text)
    # print(tag.text)

In [23]:
film_festivals_dict = {}

for fest in film_festivals_list_raw:
    fest = fest.split(",")
    film_festivals_dict[fest[0]] = fest[1]

In [24]:
# for key, value in film_festivals_dict.items():
#     print(key, "*****", value)

In [25]:
# for i in range(len(p_tags)):
#     print(i, "-", p_tags[i])

In [26]:
# Convert the dictionary to a DataFrame
df_2021 = pd.DataFrame(film_festivals_dict.items(), columns=['Festival Name', 'Raw Date'])

In [27]:
# Create empty datetime series
df_2021['Start Date'] = pd.Series(dtype=str)
df_2021['End Date'] = pd.Series(dtype=str)

In [28]:
# locate and change a specific value
df_2021.loc[1, 'Raw Date'] = "Netherlands – June 2-6"

# delete a row and reset the index
df_2021 = df_2021.drop(42, axis=0).reset_index(drop=True)

In [29]:
df_2021

,Festival Name,Raw Date,Start Date,End Date
0,Goteborg Film Festival,Sweden - January 29-February 8,NaN,NaN
1,International Film Festival Rotterdam,Netherlands – June 2-6,NaN,NaN
2,SCAD aTVfest,US - February 4-6,NaN,NaN
3,Sundance Film Festival,US - January 28-February 3,NaN,NaN
4,Glasgow Film Festival,UK - February 24-March 7,NaN,NaN
...,...,...,...,...
115,French Film Festival,UK - November 3-December 10,NaN,NaN
116,Made In Prague Festival,UK - November 7-December 10,NaN,NaN
117,Ventana Sur,Argentina - November 29-December 3,NaN,NaN
118,Red Sea Film Festival,Saudi Arabia - December 6-15,NaN,NaN


In [30]:
# df_2021.to_csv('df_2021.csv', index=True)

In [31]:
for index, row in df_2021.iterrows():
    # raw_date = row['Raw Date'].split("-–")
    raw_date = re.split(r'-|–', row['Raw Date'])
    # print(index, "*****", raw_date)
    start_date = raw_date[1].strip()
    end_date =  raw_date[2].strip()

    month = start_date.split(" ")[0]
    # print(month)

    if (len(end_date.split(" ")) < 2):
        end_date = month + " " + end_date

    start_date = start_date + " 2021"
    end_date = end_date + " 2021"

    # print(start_date, end_date)
    df_2021.at[index, 'Start Date'] = start_date
    df_2021.at[index, 'End Date'] = end_date

In [32]:
# convert the string date column to datetime objects
df_2021['Start Date'] = pd.to_datetime(df_2021['Start Date'], format='%B %d %Y')
df_2021['End Date'] = pd.to_datetime(df_2021['End Date'], format='%B %d %Y')

df_2021 = df_2021.drop('Raw Date', axis=1)

In [33]:
df_2021

,Festival Name,Start Date,End Date
0,Goteborg Film Festival,2021-01-29,2021-02-08
1,International Film Festival Rotterdam,2021-06-02,2021-06-06
2,SCAD aTVfest,2021-02-04,2021-02-06
3,Sundance Film Festival,2021-01-28,2021-02-03
4,Glasgow Film Festival,2021-02-24,2021-03-07
...,...,...,...
115,French Film Festival,2021-11-03,2021-12-10
116,Made In Prague Festival,2021-11-07,2021-12-10
117,Ventana Sur,2021-11-29,2021-12-03
118,Red Sea Film Festival,2021-12-06,2021-12-15


In [34]:
# Concatenate the two data frames
df_film_festivals = pd.concat([df_2022, df_2021, df_2020])

# Print the concatenated data frame
df_film_festivals

,Festival Name,Start Date,End Date
0,Sundance Film Festival 2022,2022-01-20 00:00:00,2022-01-30 00:00:00
1,International Film Festival Rotterdam 2022,2022-01-26 00:00:00,2022-02-06 00:00:00
2,Clermont-Ferrand International Short Film Fest...,2022-01-28 00:00:00,2022-02-05 00:00:00
3,Berlin International Film Festival 2022,2022-02-10 00:00:00,2022-02-20 00:00:00
4,Glasgow International Film Festival 2022,2022-03-02 00:00:00,2022-03-13 00:00:00
...,...,...,...
1014,"Workers Unite Film Festival, The",2020-05-08 00:00:00,2020-05-23 00:00:00
1016,Worldfest-Houston International Film & Video F...,2020-04-17 00:00:00,2020-04-26 00:00:00
1018,Yale Student Film Festival,2020-03-27 00:00:00,2020-03-29 00:00:00
1020,"YoFiFest, The Yonkers Film Festival",2020-11-06 00:00:00,2020-11-13 00:00:00


# Viewing Datasets to Merge

In [35]:
df_film_festivals

,Festival Name,Start Date,End Date
0,Sundance Film Festival 2022,2022-01-20 00:00:00,2022-01-30 00:00:00
1,International Film Festival Rotterdam 2022,2022-01-26 00:00:00,2022-02-06 00:00:00
2,Clermont-Ferrand International Short Film Fest...,2022-01-28 00:00:00,2022-02-05 00:00:00
3,Berlin International Film Festival 2022,2022-02-10 00:00:00,2022-02-20 00:00:00
4,Glasgow International Film Festival 2022,2022-03-02 00:00:00,2022-03-13 00:00:00
...,...,...,...
1014,"Workers Unite Film Festival, The",2020-05-08 00:00:00,2020-05-23 00:00:00
1016,Worldfest-Houston International Film & Video F...,2020-04-17 00:00:00,2020-04-26 00:00:00
1018,Yale Student Film Festival,2020-03-27 00:00:00,2020-03-29 00:00:00
1020,"YoFiFest, The Yonkers Film Festival",2020-11-06 00:00:00,2020-11-13 00:00:00


In [36]:
df_master_dataset

,Story Primary ID,Story ID,User Primary ID,User ID,Gender,Age,Title,Narrative,Media,Account Created Date,Interest,Associated Film Festivals
0,156581,STY1666112066,86625,USR1662300553,NaN,12,Qatar Airways cuts flights to accommodate FIFA...,"<p dir=""ltr"">Qatar Airways&#160;has withdrawn ...",https://image.pixstory.com/Pixstory-image-1666...,2022-09-04 19:39:13,"Technology, History, Food, Entertainment, Spor...",NaN
1,156580,STY1666111954,119236,USR1665943314,NaN,18,Positive thinking does not necessarily mean av...,"<p dir=""ltr"">Positive thinking does not necess...",https://image.pixstory.com/Pixstory-image-1666...,2022-10-16 23:31:54,"Technology, History, Food, Entertainment, Spor...",NaN
2,156579,STY1666111923,86566,USR1662294899,NaN,12,FIFA negotiating 'initiatives' for Qatar migra...,"<p dir=""ltr"">FIFA's president sought Monday to...",https://image.pixstory.com/Pixstory-image-1666...,2022-09-04 18:04:59,"Technology, History, Food, Entertainment, Spor...",NaN
3,156578,STY1666111798,86655,USR1662303432,NaN,12,Qatar ready for the World Cup in just over a m...,"<p dir=""ltr"">Ticket sales for the FIFA World C...",https://image.pixstory.com/Pixstory-image-1666...,2022-09-04 20:27:12,"Technology, History, Food, Entertainment, Spor...",NaN
4,156577,STY1666111463,9221,USR1632727815,male,23,"Froyo always makes everyone happy, make this h...","<p dir=""ltr""></p>\n<p dir=""ltr"">Almost all of ...",https://image.pixstory.com/Pixstory-image-1666...,2021-09-27 13:00:15,"Science, Inequality, Education, Technology, He...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
94995,95021,STY1650382324,22770,USR1642425119,male,40,Ronaldo salta la trasferta di Liverpool,Il Manchester United ha annunciato l'assenza d...,https://image.pixstory.com/Pixstory-image-1650...,2022-01-17 18:41:59,"calcio, premier league, cristiano ronaldo, man...",NaN
94996,95020,STY1650381837,32577,USR1650381467,NaN,15,Name-vishal\nFather name-Amod\nClass-7\nAge-13...,Earthday,https://image.pixstory.com/Pixstory-image-1650...,2022-04-19 20:47:47,chintan,NaN
94997,95019,STY1650381574,32498,USR1650370047,NaN,15,Earth day,Name Muskan \nClass 8\nSex f,https://image.pixstory.com/Pixstory-image-1650...,2022-04-19 17:37:27,chintan,NaN
94998,95018,STY1650380690,32574,USR1650380153,NaN,12,Name-puneet\nclass-6th\nfather name-Ramkishan\...,Earthday,https://image.pixstory.com/Pixstory-image-1650...,2022-04-19 20:25:53,chintan,NaN


In [37]:
print(type(df_film_festivals["Start Date"]))
print(type(df_master_dataset["Account Created Date"]))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


# Merging festival names into master dataset

In [38]:
# Convert date columns to datetime objects
df_master_dataset["Account Created Date"] = pd.to_datetime(df_master_dataset["Account Created Date"])
df_film_festivals["Start Date"] = pd.to_datetime(df_film_festivals["Start Date"])
df_film_festivals["End Date"] = pd.to_datetime(df_film_festivals["End Date"])

# Create a list to store the associated film festivals for each post
associated_festivals = []

# Loop through each post
for index, row in df_master_dataset.iterrows():
    # Find all film festivals where the post date falls within the festival date range
    
    # filter the dataframe based on the given date
    filtered_df = df_film_festivals[(df_film_festivals['Start Date'] <= row["Account Created Date"]) & (df_film_festivals['End Date'] >= row["Account Created Date"])]

    # extract the film festival names as a list
    film_festivals = filtered_df['Festival Name'].tolist()

    str_film_festivals = ', '.join(film_festivals)

    # Add the relevant festivals to the associated_festivals list for this post
    associated_festivals.append(str_film_festivals)

# Add the associated_festivals list as a new column in df_posts
df_master_dataset["Associated Film Festivals"] = associated_festivals


In [39]:
df_master_dataset

,Story Primary ID,Story ID,User Primary ID,User ID,Gender,Age,Title,Narrative,Media,Account Created Date,Interest,Associated Film Festivals
0,156581,STY1666112066,86625,USR1662300553,NaN,12,Qatar Airways cuts flights to accommodate FIFA...,"<p dir=""ltr"">Qatar Airways&#160;has withdrawn ...",https://image.pixstory.com/Pixstory-image-1666...,2022-09-04 19:39:13,"Technology, History, Food, Entertainment, Spor...","Venice International Film Festival 2022, Tellu..."
1,156580,STY1666111954,119236,USR1665943314,NaN,18,Positive thinking does not necessarily mean av...,"<p dir=""ltr"">Positive thinking does not necess...",https://image.pixstory.com/Pixstory-image-1666...,2022-10-16 23:31:54,"Technology, History, Food, Entertainment, Spor...",FIDBA 2022
2,156579,STY1666111923,86566,USR1662294899,NaN,12,FIFA negotiating 'initiatives' for Qatar migra...,"<p dir=""ltr"">FIFA's president sought Monday to...",https://image.pixstory.com/Pixstory-image-1666...,2022-09-04 18:04:59,"Technology, History, Food, Entertainment, Spor...","Venice International Film Festival 2022, Tellu..."
3,156578,STY1666111798,86655,USR1662303432,NaN,12,Qatar ready for the World Cup in just over a m...,"<p dir=""ltr"">Ticket sales for the FIFA World C...",https://image.pixstory.com/Pixstory-image-1666...,2022-09-04 20:27:12,"Technology, History, Food, Entertainment, Spor...","Venice International Film Festival 2022, Tellu..."
4,156577,STY1666111463,9221,USR1632727815,male,23,"Froyo always makes everyone happy, make this h...","<p dir=""ltr""></p>\n<p dir=""ltr"">Almost all of ...",https://image.pixstory.com/Pixstory-image-1666...,2021-09-27 13:00:15,"Science, Inequality, Education, Technology, He...","Haifa International Film Festival, Zurich Film..."
...,...,...,...,...,...,...,...,...,...,...,...,...
94995,95021,STY1650382324,22770,USR1642425119,male,40,Ronaldo salta la trasferta di Liverpool,Il Manchester United ha annunciato l'assenza d...,https://image.pixstory.com/Pixstory-image-1650...,2022-01-17 18:41:59,"calcio, premier league, cristiano ronaldo, man...",
94996,95020,STY1650381837,32577,USR1650381467,NaN,15,Name-vishal\nFather name-Amod\nClass-7\nAge-13...,Earthday,https://image.pixstory.com/Pixstory-image-1650...,2022-04-19 20:47:47,chintan,Pan African Film & Arts Festival 2022
94997,95019,STY1650381574,32498,USR1650370047,NaN,15,Earth day,Name Muskan \nClass 8\nSex f,https://image.pixstory.com/Pixstory-image-1650...,2022-04-19 17:37:27,chintan,Pan African Film & Arts Festival 2022
94998,95018,STY1650380690,32574,USR1650380153,NaN,12,Name-puneet\nclass-6th\nfather name-Ramkishan\...,Earthday,https://image.pixstory.com/Pixstory-image-1650...,2022-04-19 20:25:53,chintan,Pan African Film & Arts Festival 2022


In [40]:
# save dataframe to csv file
df_master_dataset.to_csv('Master_Dataset_Raw_FilmFestivals.csv', index=False)